In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Houston'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):
    
    # Set url
    url = f'https://houston.craigslist.org/search/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')
    
    print('\n================')
    print(f'This is page {i+1}')
    print('================')

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):
        
        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan
                
        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']
        # Scrape prices
        try:
            prices = link.find_all('span', class_='priceinfo')[0].text
        except:
            prices = np.nan

        # Scrape neighborhoods and handle any errors if no neighborhood found
        try:
            neighborhoods = link.find_all('div', class_='supertitle')[0].text

        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        # Ensures that the server does not get overloaded with requests
        sleep(randint(2,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
        
        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan


        amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
        # List comprehension used to make sure no empty strings get added to the list
        all_amenities = [am for am in amenities if len(am) >= 1]

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()      


This is page 1

Listing Number: 1
Square Footage: not available
Price: $1,510
Title: Lux Amenities | New Apt| Some Bad/ Low Credit Okay | FREE MOVE
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Conroe North/ Montgomery
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/conroe-lux-amenities-new-apt-some-bad/7577672540.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 2
Square Footage: not available
Price: $1,300
Title: 2610 Texas Elm Court,
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Fresno, TX
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/fresno-2610-texas-elm-court/7579687800.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 3
Square Footage: not available
Price: $278
Title: Easy freeway access, Free ba


Listing Number: 20
Square Footage: not available
Price: $1,300
Title: 10107 Bitternut Hickory Lane,
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Tomball, TX
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/tomball-bitternut-hickory-lane/7579676909.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 21
Square Footage: 927
Price: $773
Title: Washer/Dryer, Laundry Facility, Private Patio
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/baytown-washer-dryer-laundry-facility/7578014688.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 22
Square Footage: not available
Price: $1,330
Title: Spring - Bad Credit | Low Credit |Attached Garages | Repo | FREE MOV


Listing Number: 40
Square Footage: not available
Price: $1,300
Title: 17150 Amarose Drive,
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Houston, TX
Number of Bathrooms: 2.5
Link: https://houston.craigslist.org/apa/d/houston-amarose-drive/7579664669.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 41
Square Footage: not available
Price: $900
Title: 1st mo free w/12mo
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Heights/ Washington Ave
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-1st-mo-free-12mo/7579662362.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'valet parking', 'rent period: monthly']

Listing Number: 42
Square Footage: not available
Price: $1,300
Title: 17150 Amarose Drive,
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Houston, TX
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/ho


Listing Number: 61
Square Footage: 640
Price: $895
Title: Covered Parking, Home Office, Black appliances
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 7511 Beechnut Street, Houston, TX
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-covered-parking-home-office/7574448064.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no smoking', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 62
Square Footage: 687
Price: $917
Title: Reserved Parking, Gated Community, We are a ZERO Deposit Community
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 14405 Rio Bonito, Houston, TX
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-reserved-parking-gated/7570084966.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no smoking', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listin


Listing Number: 81
Square Footage: 1319
Price: $1,745
Title: 2 Bedroom Floorplans Starting At Just $1567!
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: not available
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/humble-bedroom-floorplans-starting-at/7579651797.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 82
Square Footage: 814
Price: $1,315
Title: Social Area with Billiards, Parking Garage, Online Payments Available
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: not available
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-social-area-with-billiards/7579655894.html
Amenities: ['apartment', 'w/d in unit', 'detached garage', 'rent period: daily']

Listing Number: 83
Square Footage: not available
Price: $1,450
Title: Move-in Special:$1000 off 1st full mo w/12+mo on 2BR
Date: 1/17
Number of Bedroom

...


Listing Number: 17
Square Footage: 700
Price: $925
Title: LOWER RATES AND $600 OFF JAN RENT
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Houston
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-lower-rates-and-600-off-jan-rent/7579465094.html
Amenities: ['open house dates', 'tuesday 2023-01-17', 'wednesday 2023-01-18', 'thursday 2023-01-19']

Listing Number: 18
Square Footage: 685
Price: $1,029
Title: Beautiful 1x1 $1029 with washer/dryer in unit- $700 Off Special
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Houston
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-beautiful-1x-with-washer-dryer/7579465037.html
Amenities: ['air conditioning', 'application fee details: $50 per person', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 19
Square Footage: 600
Price: $845
Title: APLIQUE HOY Y PASE A VIVIR MAÑANA
Date: 1/17
Number of Bedrooms: 1br



Listing Number: 37
Square Footage: not available
Price: $1,650
Title: Beautiful 2 Bedroom, Great Upgraded Appliances
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Galleria
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/houston-beautiful-bedroom-great/7571223720.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no parking', 'rent period: monthly']

Listing Number: 38
Square Footage: not available
Price: $1,625
Title: 2 Bedroom, Great Amenities, Washer/Dryer
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Galleria
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/houston-bedroom-great-amenities-washer/7571223357.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no parking', 'rent period: monthly']

Listing Number: 39
Square Footage: 850
Price: $1,440
Title: Double Sink Vanities, High-end Quartz Countertops, Upgraded Lighting
Date: 1/17
Number of Bedrooms: 1b


Listing Number: 58
Square Footage: 716
Price: $930
Title: Great Deals Save $$$$
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Houston
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-great-deals-save/7577873976.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'carport', 'rent period: monthly']

Listing Number: 59
Square Footage: 1687
Price: $1,695
Title: Katy Rental Homes - 575 Min Credit Score
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Katy
Number of Bathrooms: 2.5
Link: https://houston.craigslist.org/apa/d/katy-katy-rental-homes-min-credit-score/7572593478.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d hookups', 'attached garage', 'rent period: monthly']

Listing Number: 60
Square Footage: 925
Price: $1,419
Title: Private Patio or Balcony, Stove, Island Kitchen*
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Katy
Number of Bathrooms: 1
Link: https://houston.craigslist.org/


Listing Number: 79
Square Footage: 1300
Price: $1,465
Title: 1 Month Free - Downtown Houston
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Houston
Number of Bathrooms: 1
Link: https://houston.craigslist.org/apa/d/houston-month-free-downtown-houston/7572276809.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 80
Square Footage: 1550
Price: $2,000
Title: 1 - 2 Months Free Upfront
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Humble
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/humble-months-free-upfront/7573310330.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 81
Square Footage: 1550
Price: $2,000
Title: 1 - 2 Months Free Upfront
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Humble
Number of Bathrooms: 2
Link: https://houston.craigslist.org/apa/d/humble-months-


Listing Number: 101
Square Footage: 1687
Price: $1,695
Title: Rental Homes - Katy, Conroe, Spring, Willis, Kingwood, 77044 Hous77073
Date: 1/17
Number of Bedrooms: 4br
Neighborhood: Houston
Number of Bathrooms: 2.5
Link: https://houston.craigslist.org/apa/d/houston-rental-homes-katy-conroe-spring/7572330032.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d hookups', 'attached garage', 'rent period: monthly']

Listing Number: 102
Square Footage: 2039
Price: $1,725
Title: No Rent until Dec - Rental Homes- Katy, conroe, Spring, Houston -
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Conroe
Number of Bathrooms: 2.5
Link: https://houston.craigslist.org/apa/d/conroe-no-rent-until-dec-rental-homes/7574149105.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 103
Square Footage: 2039
Price: $1,550
Title: Brand New Rental Homes
Date: 1/17
Number of Bedrooms: 4br
Neighbo

In [4]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,Lux Amenities | New Apt| Some Bad/ Low Credit ...,https://houston.craigslist.org/apa/d/conroe-lu...,"$1,510",2br,NaN,Conroe North/ Montgomery,2,"[air conditioning, cats are OK - purrr, dogs a..."
1,1/17,"2610 Texas Elm Court,",https://houston.craigslist.org/apa/d/fresno-26...,"$1,300",3br,NaN,"Fresno, TX",2,"[cats are OK - purrr, dogs are OK - wooof, hou..."
2,1/17,"Easy freeway access, Free basic cable TV, Sieg...",https://houston.craigslist.org/apa/d/houston-e...,$278,1br,NaN,NaN,1,"[apartment, laundry on site, off-street parkin..."
3,1/17,"Elevator Access, On-Site Management Team, On-S...",https://houston.craigslist.org/apa/d/houston-e...,$899,0br,637,"14703 Park Row, Houston, TX",1,"[apartment, no laundry on site, off-street par..."
4,1/17,SECOND CHANCE + IMMEDIATE MOVE IN + $$$,https://houston.craigslist.org/apa/d/houston-s...,$930,1br,600,Willowbrook/ Champions/ Ella,1,"[air conditioning, furnished, apartment, w/d h..."
...,...,...,...,...,...,...,...,...,...
955,1/17,"High-speed wireless internet available, Fully ...",https://houston.craigslist.org/apa/d/houston-h...,$278,1br,NaN,NaN,1,"[apartment, laundry on site, off-street parkin..."
956,1/17,Move in Today!!,https://houston.craigslist.org/apa/d/houston-m...,"$1,064",1br,716,Houston,1,"[air conditioning, cats are OK - purrr, dogs a..."
957,1/17,Move in Today!!,https://houston.craigslist.org/apa/d/houston-m...,"$1,174",2br,851,Houston,1,"[air conditioning, cats are OK - purrr, dogs a..."
958,1/17,Move In Today!,https://houston.craigslist.org/apa/d/houston-m...,$924,1br,700,Houston,1,"[air conditioning, cats are OK - purrr, dogs a..."


In [5]:
# Exporting data to csv
df.to_csv(f'{city}_data_raw.csv', index=False)